In [34]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

### DATA READING

In [35]:
#Size of the file
os.path.getsize("/Users/meghnaijjapureddy/Downloads/archive (6)/allRNA_112RNA_corr_by6ss.csv")

2325197868

In [36]:
## Reading File using Dask

import time
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('allRNA_112RNA_corr_by6ss.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.11784076690673828 sec


In [37]:
## Reading file using Pandas

import pandas as pd
start = time.time()
df = pd.read_csv('allRNA_112RNA_corr_by6ss.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  84.68119287490845 sec


#### Dask is better than Pandas, Modin and Ray, with the least reading time

### DATA CLEANING

In [38]:
from dask import dataframe as dd
df = dd.read_csv("/Users/meghnaijjapureddy/Downloads/archive (6)/allRNA_112RNA_corr_by6ss.csv")

In [39]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, Unnamed: 0 to Bonf_p.value
dtypes: object(3), float64(5), int64(1)

In [40]:
df.head()

,Unnamed: 0,sample_size,RNA1,RNA2,corr.coeff,t_stats,p.value,BH_p.value,Bonf_p.value
0,ENSG00000121410_A1BG,4000,ENSG00000117091_CD48,ENSG00000121410_A1BG,-0.003443,0.217705,0.827670,0.912140,1.0
1,ENSG00000268895_A1BG-AS1,4000,ENSG00000117091_CD48,ENSG00000268895_A1BG-AS1,-0.006397,0.404485,0.685877,0.829000,1.0
2,ENSG00000175899_A2M,4000,ENSG00000117091_CD48,ENSG00000175899_A2M,0.000813,0.051385,0.959021,0.981338,1.0
3,ENSG00000245105_A2M-AS1,4000,ENSG00000117091_CD48,ENSG00000245105_A2M-AS1,-0.009373,0.592686,0.553425,0.735474,1.0
4,ENSG00000166535_A2ML1,4000,ENSG00000117091_CD48,ENSG00000166535_A2ML1,NaN,NaN,NaN,NaN,NaN


In [41]:
null_values = df['RNA1'].isnull()
null_count = null_values.sum().compute()

print(f"Number of null values in 'column_name': {null_count}")

Number of null values in 'column_name': 0


In [42]:

null_values2 = df['RNA2'].isnull()
null_count2 = null_values2.sum().compute()

print(f"Number of null values in 'column_name': {null_count2}")

Number of null values in 'column_name': 0


In [43]:

null_values3 = df['corr.coeff'].isnull()
null_count3 = null_values3.sum().compute()

print(f"Number of null values in 'column_name': {null_count3}")

Number of null values in 'column_name': 73564


In [44]:

null_values4 = df['t_stats'].isnull()
null_count4 = null_values4.sum().compute()

print(f"Number of null values in 'column_name': {null_count4}")

Number of null values in 'column_name': 73564


In [45]:

null_values5 = df['p.value'].isnull()
null_count5 = null_values5.sum().compute()

print(f"Number of null values in 'column_name': {null_count5}")

Number of null values in 'column_name': 73564


In [46]:

null_values6 = df['BH_p.value'].isnull()
null_count6 = null_values6.sum().compute()

print(f"Number of null values in 'column_name': {null_count6}")

Number of null values in 'column_name': 73564


In [47]:
null_values7 = df['Bonf_p.value'].isnull()
null_count7 = null_values7.sum().compute()

print(f"Number of null values in 'column_name': {null_count7}")


Number of null values in 'column_name': 73564


In [48]:
len(df.columns)

9

In [49]:
len(df.index)

14256000

In [50]:
df.head()
print(df.dtypes)
df["RNA1"].dropna()
len(df.index)
df['corr.coeff'].dropna()
len(df.index)

Unnamed: 0       object
sample_size       int64
RNA1             object
RNA2             object
corr.coeff      float64
t_stats         float64
p.value         float64
BH_p.value      float64
Bonf_p.value    float64
dtype: object


14256000

In [51]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

/var/folders/6w/ms2d338115sct6n4yjsbtm_80000gn/T/ipykernel_27615/418848558.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns=df.columns.str.replace('[#,@,&]','')


In [52]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [53]:
data=df.columns
data

Index(['Unnamed:0', 'sample_size', 'RNA1', 'RNA2', 'corr.coeff', 't_stats',
       'p.value', 'BH_p.value', 'Bonf_p.value'],
      dtype='object')

### VALIDATION

In [70]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

# Reading File  

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def matching(df,config_data):
    expected_col = list(config_data['columns'])
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Overwriting utility.py


In [76]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: allRNA_112RNA_corr_by6ss
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - unnamed_0
    - unnamed_1
    - sample_size
    - RNA1
    - RNA2
    - corr_coeff
    - t_stats
    - p_value
    - bh_p_value
    - bonf_p_value

Overwriting store.yaml


In [77]:
# Reading config file
import utility as util
config_data = util.read_config_file("store.yaml")

In [78]:
# data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'allRNA_112RNA_corr_by6ss',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['unnamed_0',
  'unnamed_1',
  'sample_size',
  'RNA1',
  'RNA2',
  'corr_coeff',
  't_stats',
  'p_value',
  'bh_p_value',
  'bonf_p_value']}

In [79]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv('/Users/meghnaijjapureddy/Downloads/archive (6)/allRNA_112RNA_corr_by6ss.csv',delimiter=',')
df_sample.head()

,Unnamed: 0,sample_size,RNA1,RNA2,corr.coeff,t_stats,p.value,BH_p.value,Bonf_p.value
0,ENSG00000121410_A1BG,4000,ENSG00000117091_CD48,ENSG00000121410_A1BG,-0.003443,0.217705,0.827670,0.912140,1.0
1,ENSG00000268895_A1BG-AS1,4000,ENSG00000117091_CD48,ENSG00000268895_A1BG-AS1,-0.006397,0.404485,0.685877,0.829000,1.0
2,ENSG00000175899_A2M,4000,ENSG00000117091_CD48,ENSG00000175899_A2M,0.000813,0.051385,0.959021,0.981338,1.0
3,ENSG00000245105_A2M-AS1,4000,ENSG00000117091_CD48,ENSG00000245105_A2M-AS1,-0.009373,0.592686,0.553425,0.735474,1.0
4,ENSG00000166535_A2ML1,4000,ENSG00000117091_CD48,ENSG00000166535_A2ML1,NaN,NaN,NaN,NaN,NaN


In [80]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "/Users/meghnaijjapureddy/Downloads/archive (6)/" + config_data['file_name'] + f'.{file_type}'

In [81]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/var/folders/6w/ms2d338115sct6n4yjsbtm_80000gn/T/ipykernel_27615/3968394673.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,Unnamed: 0,sample_size,RNA1,RNA2,corr.coeff,t_stats,p.value,BH_p.value,Bonf_p.value
0,ENSG00000121410_A1BG,4000,ENSG00000117091_CD48,ENSG00000121410_A1BG,-0.003443,0.217705,0.827670,0.912140,1.0
1,ENSG00000268895_A1BG-AS1,4000,ENSG00000117091_CD48,ENSG00000268895_A1BG-AS1,-0.006397,0.404485,0.685877,0.829000,1.0
2,ENSG00000175899_A2M,4000,ENSG00000117091_CD48,ENSG00000175899_A2M,0.000813,0.051385,0.959021,0.981338,1.0
3,ENSG00000245105_A2M-AS1,4000,ENSG00000117091_CD48,ENSG00000245105_A2M-AS1,-0.009373,0.592686,0.553425,0.735474,1.0
4,ENSG00000166535_A2ML1,4000,ENSG00000117091_CD48,ENSG00000166535_A2ML1,NaN,NaN,NaN,NaN,NaN


In [82]:
def matching(df,config_data):
    expected_col = list(config_data['columns'])
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0
#validate the header of the file
matching(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['Unnamed: 0', 'BH_p.value', 'p.value', 'corr.coeff', 'Bonf_p.value']
Following YAML columns are not in the file uploaded ['unnamed_1', 'p_value', 'unnamed_0', 'bh_p_value', 'bonf_p_value', 'corr_coeff']


0

In [74]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['Unnamed: 0', 'sample_size', 'RNA1', 'RNA2', 'corr.coeff', 't_stats',
       'p.value', 'BH_p.value', 'Bonf_p.value'],
      dtype='object')
columns of YAML are: ['unnamed_0', 'unnamed_1', 'sample_size', 'rna1', 'rna2', 'corr_coeff', 't_stats', 'p_value', 'bh_p_value', 'bonf_p_value']


In [83]:
if matching(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['Unnamed: 0', 'BH_p.value', 'p.value', 'corr.coeff', 'Bonf_p.value']
Following YAML columns are not in the file uploaded ['unnamed_1', 'p_value', 'unnamed_0', 'bh_p_value', 'bonf_p_value', 'corr_coeff']
validation failed


In [84]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('/Users/meghnaijjapureddy/Downloads/archive (6)/allRNA_112RNA_corr_by6ss.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('allRNA_112RNA_corr_by6ss.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

/Users/meghnaijjapureddy/anaconda3/lib/python3.10/site-packages/dask/dataframe/io/csv.py:771: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/Users/meghnaijjapureddy/anaconda3/lib/python3.10/site-packages/dask/dataframe/io/csv.py:771: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/Users/meghnaijjapureddy/anaconda3/lib/python3.10/site-packages/dask/dataframe/io/csv.py:771: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/Users/meghnaijjapureddy/anaconda3/lib/python3.10/site-packages/dask/dataframe/io/csv.py:771: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/Users/meghnaijjapureddy/anaconda3/lib/python3.10/site-packages/dask/dataframe/io/csv.py:771: FutureWarning: the 'line_terminator'' keyword is deprecated, use '

['/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/00.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/01.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/02.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/03.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/04.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/05.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/06.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/07.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/08.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/09.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/10.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/11.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/12.part',
 '/Users/meghnaijjapureddy/allRNA_112RNA_corr_by6ss.csv.gz/13.part',
 '/Users/meghnaijjapureddy/allRNA_

In [91]:
#number of files in gz format folder
import os
entries = os.listdir('/Users/meghnaijjapureddy/Downloads/archive (6)/')
for entry in entries:
    print(entry)

Prot-prot_corr_all.csv
Prot-RNA_corr_13gr.csv
Prot-RNA_corr_all.csv
allRNA_RNA_corr.h5
allRNA_RNA_corr.RDS
allRNA_112RNA_corr_by6ss.RDS
Prot-RNA_corr_21gr.csv
allRNA_112RNA_corr.csv
Prot-RNA_corr_63gr.csv
allRNA_112RNA_corr_by6ss.csv
Prot-prot_corr_13gr.csv


In [92]:
import os

directory = '/Users/meghnaijjapureddy/Downloads/archive (6)/'
entries = os.listdir(directory)

for entry in entries:
    if entry.endswith('.gz'):
        print(entry)


In [88]:
#size of the gz format folder
os.path.getsize('allRNA_112RNA_corr_by6ss.csv.gz')

1216